In [1]:
## Import All Needed Libraries

import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

from ray.tune import register_env, tune

from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.models import ModelCatalog
from ray.tune.logger import pretty_print

from cache_model import *
from config import get_cfg
from custom_model import *

Instructions for updating:
non-resource variables are not supported in the long term
2021-04-06 09:52:24,220	WARNING deprecation.py:34 -- DeprecationWarning: `ray.rllib.env.atari_wrappers....` has been deprecated. Use `ray.rllib.env.wrappers.atari_wrappers....` instead. This will raise an error in the future!


In [2]:
## Reuse Wrapper for AnimalAI Environment

class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "../examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig(env_config['arena_to_train'])
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [4]:
arena_configurations = ['0.yml', '1.yml', '2.yml', '3.yml', '4.yml']
int phase = 0
def on_train_result(info):
    result = info["result"]
    if result["episode_reward_mean"] > 0.9 and phase == 2:
        print("<<<<<<<<<<<<<< PHASE 3 >>>>>>>>>>>>>>>>>>")
        phase = 3
    elif result["episode_reward_mean"] > 0.85 and phase == 1:
        print("<<<<<<<<<<<<<< PHASE 2 >>>>>>>>>>>>>>>>>>")
        phase = 2
    elif result["episode_reward_mean"] > 0.82 and phase == 0:
        print("<<<<<<<<<<<<<< PHASE 1 >>>>>>>>>>>>>>>>>>")
        phase = 1
    elif result["episode_reward_mean"] > 0.80 and phase == 0:
        print("<<<<<<<<<<<<<< PHASE 0 WITH PROGRESS >>>>>>>>>>>>>>>>>>")
        phase = 0
    else:
        print("<<<<<<<<<<<<<< PHASE 0 >>>>>>>>>>>>>>>>>>")
#         phase = 0
    trainer.workers.foreach_worker(lambda ev: ev.foreach_env(
                lambda env: env.env._env.reset(arenas_configurations=
                                              ArenaConfig('../examples/configurations/curriculum/' + arena_configurations[phase]))))
    trainer.save()

## Setup configuration to use
conf = {
        "num_workers": 0,
        "env": "unity_env",
        "callbacks": {
            "on_train_result": on_train_result,
        },
        "env_config": {
            "unity_worker_id": 100,
            "arena_to_train": '../examples/configurations/curriculum/0.yml',
            },
            "model": {
                "custom_model": 'my_cnn_rnn_model',
                "custom_model_config": {},
            },
            "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "2")),
            "num_workers": 2,  # parallelism
            "framework": "torch",
            "train_batch_size": 500
        }
    
    
## Setup and register environment
ray.shutdown()
ray.init()

# Register custom models so that we can give the ID to the policy trainer
# ModelCatalog.register_custom_model("my_fc_model", MyFCForwardModel)
ModelCatalog.register_custom_model("my_rnn_model", MyRNNModel)
ModelCatalog.register_custom_model("my_convgru_model", MyConvGRUModel)  # NOTE: Only works with image observations.
ModelCatalog.register_custom_model("my_cnn_rnn_model", MyCNNRNNModel)

register_env("unity_env", lambda config: UnityEnvWrapper(config))
    
# ## Setup trainer
# trainer = PPOTrainer(config=conf, env= "unity_env")

2021-04-06 09:58:01,088	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8266


In [5]:
import ray
from ray import tune



# ray.init()
result = tune.run(
    "PPO",
    config=conf,
    resources_per_trial={
            "cpu": 5,
            "gpu": 2,
            "extra_cpu": 2,
        },
    stop={"training_iteration": 1000}
)

TuneError: Insufficient cluster resources to launch trial: trial requested 2 CPUs, 2 GPUs, but the cluster has only 8 CPUs, 0 GPUs, 6.93 GiB heap, 2.39 GiB objects (1.0 node:192.168.1.4). 

You can adjust the resource requests of RLlib agents by setting `num_workers`, `num_gpus`, and other configs. See the DEFAULT_CONFIG defined by each agent for more info.

The config of this agent is: {'num_workers': 1, 'env': 'unity_env', 'callbacks': {'on_train_result': <function on_train_result at 0x7f080fb23f80>}, 'env_config': {'unity_worker_id': 100, 'arena_to_train': '../examples/configurations/curriculum/0.yml'}, 'model': {'custom_model': 'my_cnn_rnn_model', 'custom_model_config': {}}, 'num_gpus': 2, 'framework': 'torch', 'train_batch_size': 500} 